### 3) Methodologies and Functions (S10)

So far, I have decided to forecast electricity production, consumption and import in Denmark, Greece, Ireland, Spain and Sweden because these countries' data has trend and seasonality.

My intention was to deploy **Triple Exponential Smoothing** and **Seasonal Autoregressive Integrated Moving-Average** methods for forecast.

#### 3.1) HOLT WINTERS: TRIPLE EXPONENTIAL SMOOTHING

Triple exponential smoothing is used to handle the time series data containing a seasonal component.

I need to define a **T**riple **E**xponential **S**moothing function to generate forecasts for chosen year and chosen countries with their **S**ymmetric **M**ean **A**bsolute **P**ercentage **E**rrors.

In [1]:
def tes(year):
    #Import necessary libraries    
    import os
    import itertools
    import warnings
    import numpy as np
    import pandas as pd
    from sklearn.metrics import mean_absolute_error
    from statsmodels.tsa.holtwinters import ExponentialSmoothing
    warnings.filterwarnings('ignore')
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 500)
    
    #Create a dataframe called forecasts
    forecasts = pd.DataFrame()
    #Assign production, consumption and import data folders to operations 
    operations = os.listdir("C:/Users/ismai/Desktop/electricity_forecast/electricity_data")
    
    #A for loop which executes same process below 
    for operation in operations:
        path = f"C:/Users/ismai/Desktop/electricity_forecast/electricity_data//{operation}"
        excels = os.listdir(path)
        data = pd.read_excel(os.path.join(path,excels[0])) #Here I'm reading only the first argument because I have only one.
        
        #Set year_month as index
        data.index = data['year_month']
        
        #Set columns to use and define which columns are countries
        columns = ["year_month", "Denmark", "Greece", "Ireland", "Spain", "Sweden"]
        countries = ["Denmark", "Greece", "Ireland", "Spain", "Sweden"]
        
        data = data[columns]
        
        #Create a dataframe with a copy of data
        df = data.copy()
        df['year_month'] = pd.to_datetime(df['year_month'])
        
        #A for loop that transforms all data except year_month into float.
        for col in df.columns:
            if col != 'year_month':
                df[col] = df[col].astype(float)
                
        #Set df as its year_month is always smaller than following year, because we'd like to predict with previous years. 
        df = df[df["year_month"] < str(year+1)] 
        
        #Train set for previous years, test set for chosen year
        train = df[df["year_month"] < str(year)]  
        test = df[df["year_month"] >= str(year)] 
        
        #Set hyperparameters to start at 0.10 till 1 by increasing 0.10, store them in a list called abg
        alphas = betas = gammas = np.arange(0.10, 1, 0.10)
        abg = list(itertools.product(alphas, betas, gammas))
        
        #Define a function to optimize TES
        def optimize_tes(train, test, abg, step=12): #Step: How many data point to predict? (12 months)
            print(f"Optimizing  TES parameters...for {operation}") 
            best_mae = float("inf") #Define infinity for Mean Absolute Error
            best_comb = [] #Create a list for best combination of abg
            #Create a for loop to execute same for each abg combination
            for comb in abg:
                try:
                    tes_model = ExponentialSmoothing(train, trend="add",
                                                     seasonal="add",
                                                     seasonal_periods=12). \
                        fit(smoothing_level=comb[0], #alpha argument
                            smoothing_slope=comb[1], #beta argument
                            smoothing_seasonal=comb[2]) #gamma argument

                    y_pred = tes_model.forecast(step) #Make prediction for abg
                    mae = mean_absolute_error(test, y_pred) #Calculate Mean Absolute Error
                    if mae < best_mae: #In case MAE is smaller than best_mae, update best_mae
                        best_mae = mae
                        best_comb = [round(comb[0], 2), round(comb[1], 2), round(comb[2], 2)]
                except:
                    continue
            return best_comb
        
        #Create best_combs to fill in with best combinations of abg
        best_combs = pd.DataFrame()
        #For loop to execute optimize_tes function in each column
        for col in train.columns:
            try:
                best_comb = optimize_tes(train[col], test[col], abg)
                best_combs[col] = best_comb
            except:
                continue
        
        #Drop year_month because it was datetime and the results for best combinations in this column is NaN now.
        best_combs.drop(['year_month'], axis=1, inplace=True)
        
        #Create a dataframe for optimized forecasts.
        opt_forecasts = pd.DataFrame()
        #For loop that create tes model and make predictions.
        for col in best_combs.columns:
            tes_model = ExponentialSmoothing(train[col],
                                             trend="add",
                                             seasonal="add",
                                             seasonal_periods=12).fit(smoothing_level=best_combs[col][0],
                                                                      smoothing_slope=best_combs[col][1],
                                                                      smoothing_seasonal=best_combs[col][2])

            y_pred = tes_model.forecast(12)
            opt_forecasts[col] = y_pred
        
        actual_values = test[opt_forecasts.columns].melt()["value"] #Values we already know as realized
        opt_forecasts = opt_forecasts.melt()
        opt_forecasts["model"] = 'TES'
        opt_forecasts["actual"] = actual_values.values
        opt_forecasts["operation"] = f"{operation}" #Create a column that specify if it's production, consumption or import
        opt_forecasts.columns = ["Country", "Forecast", "Model", "Actual", "Operation"] #Rename columns
        forecasts = pd.concat([forecasts, opt_forecasts],ignore_index=True)
    
    #Define a function that calculates Symmetric Mean Absolute Percentage Error
    def smape(preds, target): 
        n = len(preds)
        masked_arr = ~((preds == 0) & (target == 0))
        preds, target = preds[masked_arr], target[masked_arr]
        num = np.abs(preds - target)
        denom = np.abs(preds) + np.abs(target)
        smape_val = (200 * np.sum(num / denom)) / n
        return smape_val
    
    #Fill the forecasts' smape column with smape values and print the average
    forecasts["smape"] = [smape([forecasts["Actual"][i]], [forecasts["Forecast"][i]]) for i in range(0, forecasts.shape[0])]
    print("Average SMAPE of forecast in " + str(year) + " is: %" + str(round(forecasts['smape'].mean(),1)))
    
    #Create year_month column again and add corresponding year_month from test
    forecasts['year_month'] = "" #create an empty column
    for i in range(len(forecasts.index)):
        forecasts['year_month'][i] = test.year_month[i%12]
    
    #Reorder columns in forecasts and save it as excel document.
    forecasts = forecasts[["year_month", "Country", "Operation", "Model", "Actual", "Forecast", "smape"]]
    forecasts.to_excel(r'C:/Users/ismai/Desktop/electricity_forecast/visualization_data/' + str(year) + '-tes' + '.xlsx', index=False)
    
    return forecasts

Create a forecast file for TES 2019

In [2]:
tes(2019)

Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Average SMAPE of forecast in 2019 is: %7.9


,year_month,Country,Operation,Model,Actual,Forecast,smape
0,2019-01-01 00:00:00,Denmark,consumption,TES,3188.0,3202.501072,0.453832
1,2019-02-01 00:00:00,Denmark,consumption,TES,2766.0,2933.805339,5.888108
2,2019-03-01 00:00:00,Denmark,consumption,TES,2939.0,2987.240347,1.628025
3,2019-04-01 00:00:00,Denmark,consumption,TES,2593.0,2672.348037,3.013971
4,2019-05-01 00:00:00,Denmark,consumption,TES,2668.0,2656.298276,0.439559
5,2019-06-01 00:00:00,Denmark,consumption,TES,2474.0,2531.674333,2.304358
6,2019-07-01 00:00:00,Denmark,consumption,TES,2476.0,2479.911571,0.157855
7,2019-08-01 00:00:00,Denmark,consumption,TES,2599.0,2603.170623,0.160342
8,2019-09-01 00:00:00,Denmark,consumption,TES,2653.0,2582.561601,2.690768
9,2019-10-01 00:00:00,Denmark,consumption,TES,2721.0,2764.346556,1.580449


Create a forecast file for TES 2020

In [3]:
tes(2020)

Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for consumption
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for import
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Optimizing  TES parameters...for production
Average SMAPE of forecast in 2020 is: %8.7


,year_month,Country,Operation,Model,Actual,Forecast,smape
0,2020-01-01 00:00:00,Denmark,consumption,TES,3188.0,3303.637261,3.562653
1,2020-02-01 00:00:00,Denmark,consumption,TES,3071.0,2956.461522,3.800554
2,2020-03-01 00:00:00,Denmark,consumption,TES,2990.0,2958.292897,1.066091
3,2020-04-01 00:00:00,Denmark,consumption,TES,2596.0,2649.662636,2.045981
4,2020-05-01 00:00:00,Denmark,consumption,TES,2593.0,2576.520852,0.637550
5,2020-06-01 00:00:00,Denmark,consumption,TES,2527.0,2395.433147,5.345602
6,2020-07-01 00:00:00,Denmark,consumption,TES,2296.0,2331.374689,1.528931
7,2020-08-01 00:00:00,Denmark,consumption,TES,2606.0,2440.282281,6.567913
8,2020-09-01 00:00:00,Denmark,consumption,TES,2341.0,2431.281786,3.783590
9,2020-10-01 00:00:00,Denmark,consumption,TES,2630.0,2597.026039,1.261672


#### 3.2) SARIMA: Seasonal Autoregressive Integrated Moving-Average

Seasonal Autoregressive Integrated Moving Average, SARIMA or Seasonal ARIMA, is an extension of ARIMA that explicitly supports univariate time series data with a seasonal component.

I need to define a **S**easonal **A**uto **R**egressive **I**ntegrated **M**oving **A**verage function to generate forecasts for chosen year and chosen countries with their **S**ymmetric **M**ean **A**bsolute **P**ercentage **E**rrors.

In [4]:
def sarima(year):
    #Import necessary libraries    
    import os
    import itertools
    import warnings
    import numpy as np
    import pandas as pd
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    warnings.filterwarnings('ignore')
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 500)
    
    #Create a dataframe called forecasts
    forecasts = pd.DataFrame()
    #Assign production, consumption and import data folders to operations 
    operations = os.listdir("C:/Users/ismai/Desktop/electricity_forecast/electricity_data")
    
    #A for loop which executes same process below 
    for operation in operations:
        path = f"C:/Users/ismai/Desktop/electricity_forecast/electricity_data//{operation}"
        excels = os.listdir(path)
        data = pd.read_excel(os.path.join(path,excels[0])) #Here I'm reading only the first argument because I have only one.
        
        #Set year_month as index
        data.index = data['year_month']
        
        #Set columns to use and define which columns are countries
        columns = ["year_month", "Denmark", "Greece", "Ireland", "Spain", "Sweden"]
        countries = ["Denmark", "Greece", "Ireland", "Spain", "Sweden"]
        
        data = data[columns]
        
        #Create a dataframe with a copy of data
        df = data.copy()
        df['year_month'] = pd.to_datetime(df['year_month'])
        
        #A for loop that transforms all data except year_month into float.
        for col in df.columns:
            if col != 'year_month':
                df[col] = df[col].astype(float)
                
        #Set df as its year_month is always smaller than following year, because we'd like to predict with previous years. 
        df = df[df["year_month"] < str(year+1)] 
        
        #Train set for previous years, test set for chosen year
        train = df[df["year_month"] < str(year)]  
        test = df[df["year_month"] >= str(year)] 
                
        #Define a function for SARIMA model
        def fit_model_sarima(train, val, pdq, seasonal_pdq):
            sarima_model = SARIMAX(train, order=pdq, seasonal_order=seasonal_pdq).fit(disp=0)
            y_pred_val = sarima_model.get_forecast(steps=12)
            y_pred = y_pred_val.predicted_mean
            return mean_absolute_error(val, y_pred)
        
        #Define a function for SARIMA optimizer
        def sarima_optimizer_mae(train, val, pdq, seasonal_pdq):
            best_mae, best_order, best_seasonal_order = float("inf"), float("inf"), None
            for param in pdq:
                print(f"Optimizing SARIMA parameters...for {operation}")
                for param_seasonal in seasonal_pdq:
                    try:
                        mae = fit_model_sarima(train, val, param, param_seasonal)
                        if mae < best_mae:
                            best_mae, best_order, best_seasonal_order = mae, param, param_seasonal
                    except:
                        continue
            print('SARIMA{}x{}12 - MAE:{}'.format(best_order, best_seasonal_order, best_mae))
            return best_order, best_seasonal_order
                
        #Set hyperparameters 
        p = d = q = range(0, 2)
        pdq = list(itertools.product(p, d, q))
        seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
        
        #Create mean absolute error best orders DataFrame to fill in with best combinations of pdq
        mae_best_orders = pd.DataFrame()
        mae_best_seasonal_orders = pd.DataFrame()
        
        #For loop in countries columns to apply sarima_optimizer_mae 
        for col in countries:
            try:
                best_order, best_seasonal_order = sarima_optimizer_mae(train[col].values, test[col].values, pdq,
                                                                       seasonal_pdq)
                mae_best_orders[col] = best_order
                mae_best_seasonal_orders[col] = best_seasonal_order
            except:
                continue
        
        #Drop year_month because it was datetime and the results for best combinations in this column is NaN now.
        #mae_best_orders.drop(['year_month'], axis=1, inplace=True)
        
        #Create a dataframe for optimized forecasts.
        opt_forecasts = pd.DataFrame()
        #For loop that create tes model and make predictions.
        for col in mae_best_orders.columns:
            model = SARIMAX(train[col],
                            order=(mae_best_orders[col][0], mae_best_orders[col][1], mae_best_orders[col][2]), 
                            seasonal_order=(mae_best_seasonal_orders[col][0], mae_best_seasonal_orders[col][1], 
                                            mae_best_seasonal_orders[col][2], 12))
            
            sarima_model = model.fit(disp=0)
            pred = sarima_model.get_forecast(steps=12)
            y_pred = pred.predicted_mean
            opt_forecasts[col] = y_pred
        
        actual_values = test[opt_forecasts.columns].melt()["value"] #Values we already know as realized
        opt_forecasts = opt_forecasts.melt() 
        opt_forecasts["model"] = 'SARIMA'
        opt_forecasts["actual"] = actual_values.values
        opt_forecasts["operation"] = f"{operation}" #Create a column that specify if it's production, consumption or import
        opt_forecasts.columns = ["Country", "Forecast", "Model", "Actual", "Operation"] #Rename columns
        forecasts = pd.concat([forecasts, opt_forecasts],ignore_index=True)
                   
    #Define a function that calculates Symmetric Mean Absolute Percentage Error
    def smape(preds, target): 
        n = len(preds)
        masked_arr = ~((preds == 0) & (target == 0))
        preds, target = preds[masked_arr], target[masked_arr]
        num = np.abs(preds - target)
        denom = np.abs(preds) + np.abs(target)
        smape_val = (200 * np.sum(num / denom)) / n
        return smape_val
     
    #Fill the forecasts' smape column with smape values and print the average
    forecasts["smape"] = [smape([forecasts["Actual"][i]], [forecasts["Forecast"][i]]) for i in range(0, forecasts.shape[0])]
    print("Average SMAPE of forecast in " + str(year) + " is: %" + str(round(forecasts['smape'].mean(),1)))
    
    #Create year_month column again and add corresponding year_month from test
    forecasts['year_month'] = "" #create an empty column
    for i in range(len(forecasts.index)):
        forecasts['year_month'][i] = test.year_month[i%12]
    
    #Reorder columns in forecasts and save it as excel document.
    forecasts = forecasts[["year_month", "Country", "Operation", "Model", "Actual", "Forecast", "smape"]]
    forecasts.to_excel(r'C:/Users/ismai/Desktop/electricity_forecast/visualization_data/' + str(year) + '-sarima' + '.xlsx', index=False)
    
    return forecasts

Create a forecast file for SARIMA 2019

In [5]:
sarima(2019)

Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
SARIMA(1, 0, 0)x(0, 1, 1, 12)12 - MAE:63.94939238659049
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
SARIMA(0, 1, 0)x(1, 1, 0, 12)12 - MAE:125.123390854572
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumpt

,year_month,Country,Operation,Model,Actual,Forecast,smape
0,2019-01-01 00:00:00,Denmark,consumption,SARIMA,3188.0,3204.558454,0.518054
1,2019-02-01 00:00:00,Denmark,consumption,SARIMA,2766.0,2925.467749,5.603748
2,2019-03-01 00:00:00,Denmark,consumption,SARIMA,2939.0,2990.159868,1.725704
3,2019-04-01 00:00:00,Denmark,consumption,SARIMA,2593.0,2652.486576,2.268105
4,2019-05-01 00:00:00,Denmark,consumption,SARIMA,2668.0,2616.160619,1.962067
5,2019-06-01 00:00:00,Denmark,consumption,SARIMA,2474.0,2507.789334,1.356514
6,2019-07-01 00:00:00,Denmark,consumption,SARIMA,2476.0,2471.999796,0.161690
7,2019-08-01 00:00:00,Denmark,consumption,SARIMA,2599.0,2610.174110,0.429017
8,2019-09-01 00:00:00,Denmark,consumption,SARIMA,2653.0,2604.660809,1.838810
9,2019-10-01 00:00:00,Denmark,consumption,SARIMA,2721.0,2822.069421,3.646695


Create a forecast file for SARIMA 2020

In [6]:
sarima(2020)

Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
SARIMA(1, 1, 1)x(1, 1, 0, 12)12 - MAE:123.25254297478982
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
SARIMA(1, 1, 0)x(1, 0, 1, 12)12 - MAE:181.45848966662626
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consumption
Optimizing SARIMA parameters...for consu

,year_month,Country,Operation,Model,Actual,Forecast,smape
0,2020-01-01 00:00:00,Denmark,consumption,SARIMA,3188.0,3141.666904,1.463998
1,2020-02-01 00:00:00,Denmark,consumption,SARIMA,3071.0,2781.976548,9.876119
2,2020-03-01 00:00:00,Denmark,consumption,SARIMA,2990.0,2986.693295,0.110653
3,2020-04-01 00:00:00,Denmark,consumption,SARIMA,2596.0,2545.354070,1.970140
4,2020-05-01 00:00:00,Denmark,consumption,SARIMA,2593.0,2576.394437,0.642457
5,2020-06-01 00:00:00,Denmark,consumption,SARIMA,2527.0,2446.808874,3.224536
6,2020-07-01 00:00:00,Denmark,consumption,SARIMA,2296.0,2427.047220,5.549266
7,2020-08-01 00:00:00,Denmark,consumption,SARIMA,2606.0,2531.332200,2.906871
8,2020-09-01 00:00:00,Denmark,consumption,SARIMA,2341.0,2554.430654,8.719586
9,2020-10-01 00:00:00,Denmark,consumption,SARIMA,2630.0,2714.700057,3.169497
